In [73]:
import re
import pandas as pd

In [89]:
file = 'LocalSP96.txt'

with open(f'data/{file}', 'r', errors='ignore') as f:
    data = f.readlines()

In [90]:
# removendo linhas em branco
data = list(filter(lambda linha: linha != '\n', data))

In [91]:
# removendo cabecalho
data = list(filter(lambda linha: not linha.startswith('TRIBUNAL REGIONAL ELEITORAL - TRE'), data))
data = list(filter(lambda linha: not linha.startswith('SECRETARIA DE INFORMATICA   - SI'), data))
data = list(filter(lambda linha: not linha.startswith('SISTEMA DE ALISTAMENTO ELEITORAL'), data))
data = list(filter(lambda linha: not linha.__contains__('LOCAIS DE VOTACAO'), data))
data = list(filter(lambda linha: linha != '------------------------------------------------------------------------------\n', data))


In [92]:
# removendo quebra de linha
data = list(map(lambda linha: linha[:-1], data))

In [93]:
data

['ZONA : 0001     MUNICIPIO: 71072 - SAO PAULO',
 'LOCAL: 1015-COLEGIO OBJETIVO',
 'END. :           AV. PAULISTA, 900            BELA VISTA         000000',
 'CEP  : 00000000   No. POSSIVEL SECOES:   30',
 'SECOES/QUANTIDADE DE ELEITORES',
 '0001/0522  0002/0533  0003/0509  0004/0547  0005/0540  0006/0510  0007/0502',
 '0008/0495  0009/0492  0010/0493  0011/0495  0012/0484  0013/0488  0014/0500',
 '0015/0515  0016/0546  0017/0500  0018/0488  0019/0484  0020/0514  0021/0485',
 '0022/0483  0023/0485  0024/0487  0025/0480  0026/0485  0027/0475  0355/0479',
 '0361/0485',
 'No. REAL SECOES/ELEITORES: 0029/14501',
 'LOCAL: 1023-EEPG RODRIGUES ALVES',
 'END. :           AV. PAULISTA, 227            BELA VISTA         000000',
 'CEP  : 00000000   No. POSSIVEL SECOES:   23',
 'SECOES/QUANTIDADE DE ELEITORES',
 '0029/0478  0030/0499  0031/0499  0032/0491  0033/0458  0034/0437  0035/0463',
 '0036/0477  0037/0454  0038/0443  0039/0436  0040/0442  0041/0434  0042/0434',
 '0043/0444  0044/0437  004

In [94]:
# Iterando sobre cada linha

resultado = []
padrao_cep = re.compile("[0-9]{5}[-]{0,1}[0-9]{3}")
padrao_secao_eleitores = re.compile("[0-9]{4}[/][0-9]{4}")
valida_secoes, valida_eleitores = [], []

for i in data:
    if i.startswith('ZONA'):
        zona = i[7:11]
        cod_municipio = i.split(' - ')[0].split(': ')[-1]
        municipio = i.split(' - ')[-1]

    if i.startswith('LOCAL'):
        cod_local = i.split(': ')[-1].split('-')[0]
        local = i.split(': ')[-1].split('-')[1]

    if i.startswith('END'):
        endereco = i.split(':')[-1].lstrip()

    # if i.startswith('Bairro    :'):
    #     bairro = i.split(':')[-1].lstrip()

    if i.startswith('CEP'):
        cep = padrao_cep.search(i).group()

    if i.startswith('No. REAL SECOES/ELEITORES'):
        valida_secoes.append(int(i.split(':')[-1].lstrip().split('/')[0]))
        valida_eleitores.append(int(i.split(':')[-1].lstrip().split('/')[1]))
    else:
        secao_eleitores = padrao_secao_eleitores.findall(i)
        for s in secao_eleitores:
            secao = s.split('/')[0]
            eleitores = int(s.split('/')[1])
            resultado.append([cod_municipio, municipio, cod_local, local, endereco, cep, zona, secao, eleitores])


In [95]:
df_locais_votacao = pd.DataFrame(resultado, columns=['cod_municipio', 'municipio', 'cod_local', 'local', 'endereco', 'cep', 'zona', 'secao', 'eleitores'])

In [96]:
df_locais_votacao

,cod_municipio,municipio,cod_local,local,endereco,cep,zona,secao,eleitores
0,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",00000000,0001,0001,522
1,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",00000000,0001,0002,533
2,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",00000000,0001,0003,509
3,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",00000000,0001,0004,547
4,71072,SAO PAULO,1015,COLEGIO OBJETIVO,"AV. PAULISTA, 900 BELA VISTA ...",00000000,0001,0005,540
...,...,...,...,...,...,...,...,...,...
43613,65633,ITAQUAQUECETUBA,1120,EEPG ALCEU MAGALHAES COUTINHO,"AV. GONCALVES DIAS, 350 PARQUE MARENGO ...",00000000,0377,0154,580
43614,65633,ITAQUAQUECETUBA,1120,EEPG ALCEU MAGALHAES COUTINHO,"AV. GONCALVES DIAS, 350 PARQUE MARENGO ...",00000000,0377,0161,578
43615,65633,ITAQUAQUECETUBA,1120,EEPG ALCEU MAGALHAES COUTINHO,"AV. GONCALVES DIAS, 350 PARQUE MARENGO ...",00000000,0377,0168,580
43616,65633,ITAQUAQUECETUBA,1228,E E P S G AMALIA MARIA DOS SANTOS,RUA VITAL BRASIL VILA MA...,08570000,0377,0170,594


In [97]:
sum(valida_secoes)

43618

In [98]:
sum(valida_eleitores)

22100788

In [99]:
df_locais_votacao.eleitores.sum()

22100788

In [100]:
df_locais_votacao.to_csv('resultados/locais_de_votacao-1996.csv')